In [57]:
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install opustools
!pip install lingtrain_aligner
!pip install razdel
!pip install dataparser
!pip install dateparser
!pip install sentence_transformers
!pip3 install nltk
!pip install xmltodict
!pip install pypinyin
!pip install pykakasi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
from sentence_transformers import SentenceTransformer, SentencesDataset, LoggingHandler, losses
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import SentenceEvaluator
from torch.utils.data import DataLoader
from datetime import datetime
import math, re
from transformers import MBartForConditionalGeneration as Model
from transformers import MBart50TokenizerFast as Tokenizer
import datasets
import torch
from random import choice
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
from xml.dom.minidom import parse
import opustools
from lingtrain_aligner import splitter, aligner, resolver, metrics
import nltk
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu

# 1984, Orwell

In [ ]:
! unzip MTE1984-ana.zip

In [ ]:
def parse_alignment(alignment, t1):
    tree = parse(alignment)
    first_lang, second_lang = [], []

    links = tree.getElementsByTagName("link")
    for link in links:
        targets = dict(link.attributes.items())['targets'].split()
        s1, s2 = [], []
        for tgt in targets:
            ts = tgt.split("#")
            # if ts[0] == 'oana-en.xml':
            if ts[0] == t1:
                s1.append(ts[1])
            else:
                s2.append(ts[1])
        first_lang.append(s1)
        second_lang.append(s2)

    return first_lang, second_lang

def parse_lang_xml(filename, align_list):
    line_num, cur_str, strr, strings = 0, 0, "", []
    tree = parse(filename)
    paragrs = tree.getElementsByTagName("p")
    for p in paragrs:
        for st in p.childNodes:
            if st.nodeType is not node.TEXT_NODE:
                s = ""
                # print(st)
                if st.nodeType is not node.TEXT_NODE:
                    # print(dict(st.attributes.items()))
                    str_id = dict(st.attributes.items())['xml:id']
                    for w in st.childNodes:
                        if w.nodeType is not node.TEXT_NODE:
                            for n in w.childNodes:
                                s += n.data
                                s += " "
                    # print(s)
                    if line_num < len(align_list) and str_id not in align_list[line_num] and cur_str > 0:
                        strings.append(strr)
                        # print(strr)
                        strr = ""
                        cur_str = 0
                        if line_num < len(align_list):
                            line_num += 1
                    if line_num < len(align_list) and str_id not in align_list[line_num] and cur_str == 0:
                        s = ""
                    if line_num < len(align_list) and len(align_list[line_num]) == 1 and align_list[line_num][0] == str_id:
                        if line_num < len(align_list):
                            line_num += 1
                        # print(s)
                        strings.append(s)
                    elif line_num < len(align_list) and len(align_list[line_num]) > 1 and str_id in align_list[line_num]:
                        strr += s
                        cur_str += 1
    return strings

def write_res(strings, filename):
    of = open(filename, "w")
    for s in strings:
        of.write(s)
        of.write("\n")
    of.close()

def parse_xml(alignment, lang1, lang2, t1, t2, xml1, xml2):
    first_lang, second_lang = parse_alignment(alignment, t1)
    print(len(first_lang))
    print(len(second_lang))

    # parse xmls according to the alignment
    strings = parse_lang_xml(xml1, first_lang)
    write_res(strings, lang1)

    strings = parse_lang_xml(xml2, second_lang)
    write_res(strings, lang2)

In [ ]:
# parse_xml('/content/MTE1984-ana/oalg-enro.xml', '/content/en1.txt', '/content/ro1.txt', 'oana-en.xml', 'oana-ro.xml', "/content/MTE1984-ana/oana-en.xml", "/content/MTE1984-ana/oana-ro.xml")
# parse_xml('/content/MTE1984-ana/oalg-bgen.xml', '/content/en.txt', '/content/bg.txt', 'oana-en.xml', 'oana-bg.xml', "/content/MTE1984-ana/oana-bg.xml", "/content/MTE1984-ana/oana-en.xml")
# parse_xml('/content/MTE1984-ana/oalg-enhu.xml', '/content/en.txt', '/content/hu.txt', 'oana-en.xml', 'oana-hu.xml', "/content/MTE1984-ana/oana-en.xml", "/content/MTE1984-ana/oana-hu.xml")

# Jane Air

**extraction**

In [ ]:
# !opus_read --directory Books --source de --target es -m 5500 --write tmp.txt

In [ ]:
book = open('/content/tmp.txt', "r")
cleaned = open('/content/tmp1.txt', "w")
lines = book.readlines()

sources, targets = [], []

s_s = ""
s_t = ""

for line in lines:
    line = line.strip('\n')
    if '(src)' in line:
        s_s += line.split('>')[1]
        s_s += " "
    elif '(trg)' in line:
        s_t += line.split('>')[1]
        s_t += " "
    elif '==' in line:
        sources.append(s_s)
        targets.append(s_t)
        s_s = ""
        s_t = ""

write_res(sources, '/content/de-es.de')
write_res(targets, '/content/de-es.es')

**Alignment**

In [ ]:
# text1, text2, text3, text4 are vars with text

In [ ]:
model = SentenceTransformer('LaBSE')

lang_from = "de"
lang_to = "es"
db_path = "alignment.db"
# lang_from = "de"
# lang_to = "en"

# splitted_from = splitter.split_by_sentences(text1.split('\n'), lang_from)
# splitted_to = splitter.split_by_sentences(text2.split('\n'), lang_to)
aligner.fill_db(db_path, lang_from, lang_to, splitted_from, splitted_to)

steps = 3

for i in range(steps):
    conflicts, rest = resolver.get_all_conflicts(db_path,
                        min_chain_length=2+i,
                        max_conflicts_len=6*(i+1),
                        batch_id=-1)

    resolver.resolve_all_conflicts(db_path, conflicts, "sentence_transformer_multilingual_labse", show_logs=False)

    if len(rest) == 0:
        break

output_path="/content"

saver.save_plain_text(db_path, os.path.join(output_path, f"corpora_{lang_from}.txt"), direction="from", batch_ids=[])
saver.save_plain_text(db_path, os.path.join(output_path, f"corpora_{lang_to}.txt"), direction="to", batch_ids=[])


In [ ]:
aligner.align_db(db_path,
                model_name="sentence_transformer_multilingual_labse",
                batch_size=200,
                window=50,
                batch_ids=[],
                save_pic=False,
                embed_batch_size=5,
                normalize_embeddings=True,
                show_progress_bar=True,
                shift=0)

**Translation**

In [ ]:
def translate(text, refs, lang, md):
    tokenizer.src_lang = lang
    result = []
    res_list = []
    refs_list = []
    
    for i, t in enumerate(text):
        if i % 10 == 0:
            print(i)
        encoded = tokenizer(t, return_tensors="pt")
        if md == 0:
            generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
        else:
            generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
        res = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        
        res_list.append(re.sub("[^\w]", " ",  res[0]).split())
        refs_list.append([re.sub("[^\w]", " ",  refs[i]).split()])
        result.append(res[0])
    return res_list, refs_list, result

In [55]:
def count_bleu(lang1, lang2, mod, rs, hs):
    refs = open(rs, "r").readlines() # human translation
    hyps = open(hs, "r").readlines() # machine translation

    refs_list, hyps_list = [], []

    s = ""
    s += lang1 
    s += " - " 
    s += lang2 
    s += " - " 
    s += mod
    print(s)

    for i, ref in enumerate(refs):
        # print([re.sub("[^\w]", " ",  refs[i]).split()])
        refs_list.append([re.sub("[^\w]", " ",  refs[i]).split()])
        hyps_list.append(re.sub("[^\w]", " ",  hyps[i]).split())
        smoothie = SmoothingFunction().method4
    score = nltk.translate.bleu_score.corpus_bleu(refs_list, hyps_list)
    print(score)

In [ ]:
def prepare_samples(sample_orig, sample_eup_tr):
    orig_sample = open(sample_orig, "r")
    tr_eu_sample = open(sample_eup_tr, "r")
    
    sample = orig_sample.readlines()
    sample_refs = tr_eu_sample.readlines()
    
    orig_sample.close()
    tr_eu_sample.close()
        
    return sample, sample_refs

In [ ]:
def tr_and_write(lang, trans, md, sample, sample_refs):
    rsl, rfl, rs = translate(sample, sample_refs, lang, md)
    trans_sample = open(trans, "w")
    for r in rs:
        trans_sample.write(r)
        trans_sample.write("\n")
    trans_sample.close()

In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")    
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

sample1, sample_refs1 = prepare_samples("de-es_ja.es", "de-es_ja.de")
sample2, sample_refs2 = prepare_samples("de-en_ja.en", "de-en_ja.de")

print("es-de")
tr_and_write("es", "es_translated_m2m.txt", 0, sample1, sample_refs1)

print("en-de")
tr_and_write("en", "en_translated_m2m.txt", 0, sample2, sample_refs2)

model = Model.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")    
tokenizer = Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

print("es-de")
result = count_bleu("es_XX", "es_translated_bart.txt", 1, sample1, sample_refs1)
print(result['bleu'])

print("en-de")
result = count_bleu("en_XX", "es_translated_bart.txt", 1, sample2, sample_refs2)
print(result['bleu'])

In [56]:
count_bleu("en", "de", "m2m", "/content/de-en_ja.de", "/content/en_translated_m2m.txt")
count_bleu("es", "de", "m2m", "/content/de-es_ja.de", "/content/es_translated_m2m.txt")
count_bleu("en", "de", "mBart", "/content/de-en_ja.de", "/content/en_translated_bart.txt")
count_bleu("es", "de", "mBart", "/content/de-es_ja.de", "/content/es_translated_bart.txt")

en - de - m2m
0.07118307490841326
es - de - m2m
0.014667333605911407
en - de - mBart
0.08086458114310201
es - de - mBart
0.008181394008549434
